## Load Packages

In [1]:
import random
import numpy as np
import torch
import pickle
import pandas as pd
import os
from tqdm import tqdm_notebook
from nhps.models import nhp
from nhps.io import processors
from nhps.miss import factorized

## Functions

In [2]:
## function to convert pkl to df
def convert_dict_to_df(pkl):
    df = pd.DataFrame()
    for i in tqdm_notebook(range(0,len(pkl))):
        temp_df = (pd.DataFrame(pkl[i]))
        temp_df['seq_id'] = i
        df = pd.concat([df,temp_df])
    return df.reset_index(drop=True)

### Function to load dataset

In [3]:
def load_dataset(**args):
    '''
        'PathData': 'Directory where data is stored'
        'Split': Type of data to use ['Train','Test','dev']. While testing choose either test or dev
        'Small': reduce size of data for debug runs
    '''
    data_path = os.path.join(args['PathData'], args['Split'] + '.pkl')
    with open(data_path, 'rb') as fp:
        pkl_dev = pickle.load(fp)
    data_dev, data_dev_gold = pkl_dev['seqs_obs'], pkl_dev['seqs']

    if args['Small'] > -1:
        data_dev = data_dev[:args['Small']]
        data_dev_gold = data_dev_gold[:args['Small']]

    #obs_num, unobs_num = pkl_dev['obs_num'], pkl_dev['unobs_num']
    #total_event_num = obs_num + unobs_num
    total_event_num = pkl_dev['total_num']

    return [data_dev, data_dev_gold], [total_event_num]
    #return [data_dev, data_dev_gold], [obs_num, unobs_num, total_event_num]


### Function to Propose Particles

In [4]:
def propose_particles(**args):
    '''
        Model : nhpf or nhps
        Seed : 12345 or any other number
        UseGPU : True        
     'PathData':'data/pilottaxi'
     DimLSTM : 16
        
    '''
#     handler = args['handler']# What is handler. Assuming that it is used for multi processing
#     handler.print("Propose particles for model {}".format(args['Model']))
    print("Propose particles for model {}".format(args['Model']))
    random.seed(args['Seed'])
    np.random.seed(args['Seed'])
    torch.manual_seed(args['Seed'])

    [data_dev, data_dev_gold], [total_event_num] = load_dataset(**args)
    # seq_bases are the x's.
    # We may use miss_mec to generate seq_bases in the future.
    seq_bases = data_dev

    if not args['UseGPU']:
        device = 'cpu'
    else:
        device = 'cuda'

    sampling = 1

    miss_mec = factorized.FactorizedMissMec(
        device = 'cuda' if args['UseGPU'] else 'cpu',
        config_file = os.path.join(args['PathData'], 'censor.conf')
    )

    proc = processors.DataProcessorNeuralHawkes(
        idx_BOS=total_event_num,
        idx_EOS=total_event_num+1,
        idx_PAD=total_event_num+2,
        miss_mec=miss_mec,
        sampling=sampling,
        device = 'cuda' if args['UseGPU'] else 'cpu'
    )

    if args['Model'] == 'nhps':
        agent = nhp.NeuralHawkes(
            total_num=total_event_num, hidden_dim=args['DimLSTM'],
            device=device, miss_mec=miss_mec
        )
        agent.initBackwardMachine(hidden_dim_back=args['BackDimLSTM'], type_back=args['BackType'],
                                  back_beta=args['BackBeta'])
    elif args['Model'] == 'nhpf':
        agent = nhp.NeuralHawkes(
            total_num=total_event_num, hidden_dim=args['DimLSTM'],
            device=device, miss_mec=miss_mec
        )
    else:
        raise NotImplementedError

    agent.load_state_dict(
        torch.load(args['final_model_location'], map_location='cpu') )

    # NOTE: Here I just simply replace neglect_mask with r vector
    # In a more general case mentioned in the paper,
    # r factor should take the whole history into account.
    # But in our experiment, events are censored independently,
    # so the r vector is constant for all the events that're going to be proposed.
    # I set history as None to get the r vector.
    # **Reverse Changes**
    agent.setMaskIntensity(miss_mec.neglect_mask())

    if args['UseGPU']:
        agent.cuda(device)
    agent.eval()

    input_dev = []
    input_dev_with_grountruth = []
    all_weights = list()
    all_particles = list()
    all_log_proposals = list()
    all_num_unobs = list()

    for i_dev, (one_seq_dev, one_seq_dev_gold) in enumerate(zip(data_dev, data_dev_gold)):

        one_seq_dev_augmented = proc.orgSeq(
            one_seq_dev, one_seq_dev_gold[-1]['time_since_start']
        )
        input_dev.append(
            proc.augmentLogProbMissing(
                agent.sample_particles(
                    args['NumParticle'], one_seq_dev_augmented, args['NumUnobserved'],
                    args['Multiplier'],
                    resampling=args['Resampling'],
                    need_eliminate_log_base=args['EliminateBase'] )))

        input_dev_with_grountruth.append(
            proc.processSeq(
                one_seq_dev_gold, n=1, seq_obs=one_seq_dev))

        if (i_dev+1) % args['SizeBatch'] == 0 or \
                (i_dev == len(data_dev)-1 and (len(input_dev)%args['SizeBatch']) > 0):

            r"""
            this part is computing log q(z | x)
            where z is ground truth (similar to train nhps)
            """
            batch_seqs_with_groundtruth = proc.processBatchSeqsWithParticles(
                input_dev_with_grountruth)
            log_proposals, num_unobs = agent(batch_seqs_with_groundtruth, mode=9)
            all_log_proposals.append(log_proposals.detach().cpu().numpy())
            all_num_unobs.append(num_unobs.detach().cpu().numpy())
            input_dev_with_grountruth = list()

            r"""
            this part is computing weights for decoding
            """

            # If resampling is on, we could directly get the weights of particles.
            if args['Resampling']:
                log_weights = np.array([input_[2].cpu().detach().numpy() for input_ in input_dev])
                unnormalized_weights = np.exp(log_weights)
                weights = (unnormalized_weights.T / unnormalized_weights.sum(axis=1)).T
            else:
                # If not, we could use our old method.
                batch_seqs_dev = proc.processBatchSeqsWithParticles(input_dev)
                weights, _ = agent(batch_seqs_dev, mode=4)
                weights = weights.detach().cpu().numpy()

            weights_orders = np.argsort(-weights, axis=1)

            for i_batch, [event_, dtime_, _, _, len_seq_, _, _, _, _, _, _, _] in enumerate(input_dev):
                particles = list()
                all_particles.append(particles)
                for particle_idx in weights_orders[i_batch]:
                    event, dtime, len_seq = event_[particle_idx], \
                                            dtime_[particle_idx], len_seq_[particle_idx]
                    particles.append(proc.getSeq(event.cpu(), dtime.cpu(), len_seq.cpu()))
                all_weights.append(weights[i_batch][weights_orders[i_batch]])

            input_dev = []
#             handler.print("Proposing {}-th {} seq".format(i_dev+1, args['Split']))
            print("Proposing {}-th {} seq".format(i_dev+1, args['Split']))

    all_weights = np.array(all_weights)
    all_log_proposals = np.concatenate(all_log_proposals)
    all_num_unobs = np.concatenate(all_num_unobs)

    # Note that particles in each batch are ordered by the their weights inversely.
    return all_weights, all_particles, seq_bases, all_log_proposals, all_num_unobs,log_proposals, num_unobs,weights_orders

## Run propose_particles()

In [5]:
args = {
    'PathData':'data/test',
    'Split':'Test',
    'NumUnobserved':	1, 
    'Multiplier':	1,
    'NumParticle':	1,
    'Cost':	1,
    'MultiplierCost':	2,
    'NumCost':	1,
    'Split':	'dev',
    'SizeBatch':	1,
    'UseGPU':	True,
    'Seed':	12345,
    'Small':	3, # change value to increase the number of output sequences
    'MaxIter':	5,
    'ProcessPerDevice':	2,
#     'VisibleDevice':	
    'MultiProcess':	None,
    'Resampling':	True,
    'EliminateBase':	True,
    'Model':'nhps',
    'PathModel':'./',
    'BackDimLSTM':16,
    'DimLSTM':16,
    'BackType':'add',
    'BackBeta':1,
    'final_model_location':'./logs/saved_model'
}

In [6]:
all_weights, all_particles, seq_bases, all_log_proposals, all_num_unobs,log_proposals, num_unobs,weights_orders = propose_particles(**args)

Propose particles for model nhps
Proposing 1-th dev seq
Proposing 2-th dev seq
Proposing 3-th dev seq


### all_weights

In [7]:
[data_dev, data_dev_gold], [total_event_num] = load_dataset(**args)

In [8]:
data_dev = convert_dict_to_df(data_dev)
data_dev_gold = convert_dict_to_df(data_dev_gold)

C:\Users\am21907\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [9]:
data_dev.head()

,patient_id,idx_event,type_event,time_since_start,time_since_last_event,seq_id
0,7.172600e+12,1.0,11.0,0.0,0.0,0
1,7.172600e+12,2.0,2.0,7.0,7.0,0
2,7.172600e+12,3.0,11.0,79.0,72.0,0
3,7.172600e+12,4.0,2.0,154.0,75.0,0
4,7.172600e+12,5.0,11.0,189.0,35.0,0


In [10]:
data_dev['type_event'].max()

18.0

In [11]:
data_dev_gold['type_event'].max()

18.0

### all_particles (output sequence)

all_particles[0] holds the output sequence. <br>
<b> Note </b>: Number of imputed events is decided by the NumUnobserved

In [25]:
len(seq_bases)==args['Small']

True

In [26]:
convert_dict_to_df(all_particles[0])

C:\Users\am21907\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,type_event,time_since_last_event,time_since_start,seq_id
0,11,0.000000,0.000000,0
1,2,7.000000,7.000000,0
2,8,0.723843,7.723843,0
3,11,71.276154,79.000000,0
4,2,75.000000,154.000000,0
...,...,...,...,...
135,4,27.989990,1774.000000,0
136,4,21.000000,1795.000000,0
137,4,28.000000,1823.000000,0
138,1,28.000000,1851.000000,0


In [27]:
convert_dict_to_df(all_particles[1])

C:\Users\am21907\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,type_event,time_since_last_event,time_since_start,seq_id
0,4,0.000000,0.000000,0
1,8,2.513605,2.513605,0
2,4,58.486397,61.000000,0
3,7,25.000000,86.000000,0
4,9,8.000000,94.000000,0
...,...,...,...,...
80,10,0.010010,1784.020020,0
81,4,7.979980,1792.000000,0
82,1,10.000000,1802.000000,0
83,4,17.000000,1819.000000,0


In [28]:
len(all_particles[0][0]) - len(seq_bases[0])

1

In [29]:
len(all_particles[1][0]) - len(seq_bases[1])

1

### seq_base

Base sequences ( events without imputation)

In [30]:
len(seq_bases)==args['Small']

True

In [31]:
convert_dict_to_df([seq_bases[0]])

C:\Users\am21907\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,patient_id,idx_event,type_event,time_since_start,time_since_last_event,seq_id
0,7.172600e+12,1.0,11.0,0.00,0.00,0
1,7.172600e+12,2.0,2.0,7.00,7.00,0
2,7.172600e+12,3.0,11.0,79.00,72.00,0
3,7.172600e+12,4.0,2.0,154.00,75.00,0
4,7.172600e+12,5.0,11.0,189.00,35.00,0
...,...,...,...,...,...,...
134,7.172600e+12,135.0,4.0,1774.00,28.00,0
135,7.172600e+12,136.0,4.0,1795.00,21.00,0
136,7.172600e+12,137.0,4.0,1823.00,28.00,0
137,7.172600e+12,138.0,1.0,1851.00,28.00,0


In [32]:
convert_dict_to_df([seq_bases[1]])

C:\Users\am21907\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,patient_id,idx_event,type_event,time_since_start,time_since_last_event,seq_id
0,7.172604e+12,1.0,4.0,0.00,0.00,0
1,7.172604e+12,2.0,4.0,61.00,61.00,0
2,7.172604e+12,3.0,7.0,86.00,25.00,0
3,7.172604e+12,4.0,9.0,94.00,8.00,0
4,7.172604e+12,5.0,4.0,100.00,6.00,0
...,...,...,...,...,...,...
79,7.172604e+12,80.0,10.0,1784.02,0.02,0
80,7.172604e+12,81.0,4.0,1792.00,8.00,0
81,7.172604e+12,82.0,1.0,1802.00,10.00,0
82,7.172604e+12,83.0,4.0,1819.00,17.00,0


### all_log_proposals

An indicator showing how well the new series with imputed events fits the distribution

In [33]:
len(all_log_proposals)==args['Small']

True

In [34]:
all_log_proposals

array([-25550.045, -25403.438, -18917.43 ], dtype=float32)

### all_num_unobs

This array seems to hold the number of events that were originally observed in the corresponding sequence

In [35]:
all_num_unobs

array([161.,   0., 189.], dtype=float32)

If hypothesis is true, then the difference between the all_num_unobs and it's corresponding output sequence will by = args['NumUnobserved']

In [36]:
for i,num in enumerate(all_num_unobs):
    print(len(all_particles[i][0]) - num == args['NumUnobserved'])

False
False
False


In [37]:
len(all_particles[1][0])

85